In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']

lst_soorten_meta = ['Metaal',
 'Artefact',
 'Glas',
 'Stelling',
 'Hout',
 'Spijker',
 'Plaatsing',
 'Keramiek',
 'Put',
 'Vondst',
 'Standplaats',
 'Aardewerk',
 'Munt',
 'Muur',
 'Skelet',
 'Doos',
 'Project',
 'Spoor',
 'Vindplaats',
 'Leer',
 'Steen',
 'Bot',
 'Vlak',
 'Kleipijp']
df_soorten_meta = pd.DataFrame(lst_soorten_meta).rename(columns={0: 'Soort'}) 

In [3]:
grp_aggr = [{'$group': {'_id': {'project': "$brondata.project", 'soort': "$soort"},'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'soort': "$_id.soort", 'teller': { '$ifNull': ["$teller", 0]}}}}]
grp_aggr_artef = [{"$match": {"soort": 'Artefact'}}
                 ,{'$group': {'_id': {'project': "$brondata.project", 'soort': "$artefactsoort"},'teller': {"$sum": 1}}}
                 ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'soort': "$_id.soort", 'teller': { '$ifNull': ["$teller", 0]}}}}]

def AggregateData(df, fase): 
    df['teller'] = pd.to_numeric(df['teller'])
    df = df.groupby('soort').agg({'project':lambda x: list(x), 'teller':lambda x: sum(x)})
    df['Count'] = df.apply(lambda x: len(x.project), axis=1)
    df['Stage'] = fase
    df = df.sort_values('teller', ascending=False).reset_index()
    df = df.rename(columns={'soort': 'Soort', 'project': 'Projecten', 'teller': 'Aantal_Records', 'Count': 'Aantal_projecten'}).sort_index()
    return df

def getData(collection, soort): 
    df1 = pd.DataFrame(list(collection.aggregate(grp_aggr_artef)))
    df2 = pd.DataFrame(list(collection.aggregate(grp_aggr)))

    return AggregateData(df1.append(df2[df2.soort != 'Artefact']), soort)

In [4]:
df_singlestore = getData(analyseCol, 'SingleStore')
df_singlestore.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Standplaats,[MAGAZIJN],16814,1,SingleStore
1,Aardewerk,"[DC22, DC05, DC97, DB34, DC21, DC36, DC20, DC7...",14091,9,SingleStore
2,Plaatsing,[MAGAZIJN],11268,1,SingleStore
3,Doos,[MAGAZIJN],5546,1,SingleStore
4,Vondst,"[DC97, DC24_STADSKANTOOR, DC154, DC112, DC11, ...",5442,11,SingleStore


In [5]:
df_singlestoreclean = getData(analyseColClean, 'SingleStoreClean')
df_singlestoreclean.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Standplaats,[MAGAZIJN],16814,1,SingleStoreClean
1,Aardewerk,"[DC22, DC05, DC97, DB34, DC21, DC36, DC20, DC7...",14091,9,SingleStoreClean
2,Plaatsing,[MAGAZIJN],11268,1,SingleStoreClean
3,Doos,"[MAGAZIJN, DC21, DC112, DC97, DC11, DB34, DC18...",5574,8,SingleStoreClean
4,Vondst,"[DC72, DC14, DB8, DC9, DC97, DC23, DC154, DC93...",5538,22,SingleStoreClean


In [6]:
lst_tables = ['Def_Project', 'Def_Put', 'Def_Vondst', 'Def_Spoor', 'Def_Stelling', 'Def_Doos', 'Def_Standplaats', 'Def_Plaatsing', 'Def_Vlak', 'Def_Vindplaats', 'Def_Artefact'] #['Def_Project', 'Def_Vondst', 'Def_Stelling', 'Def_Plaatsing', 'Def_Vindplaats', 'Def_Artefact']
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object

def getProject(brondata):    
    #print(brondata)
    if brondata is None or brondata != "":
        return regexProject.search(brondata).group(1)
    else: 
        return "" 

df_tables_projects = pd.DataFrame()
    
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    for table in lst_tables:     
        df = pd.read_sql_query('SELECT brondata from "' + table + '"', connection)
        
        df_out = pd.DataFrame()
        df_out['project'] = df.apply(lambda x: getProject(x['brondata']), axis=1)
        df_out['soort'] = table[4:]
        df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(df_out).groupby(['project', 'soort']).size().reset_index(name='teller')])

with engine.connect() as connection:
    df = pd.read_sql_query('SELECT brondata, artefactsoort from "Def_Artefact"', connection)

    df_out = pd.DataFrame()
    df_out['project'] = df.apply(lambda x: getProject(x['brondata']), axis=1)
    df_out['soort'] = df['artefactsoort']
    df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(df_out).groupby(['project', 'soort']).size().reset_index(name='teller')])

        
df_doelsysteem = AggregateData(df_tables_projects, 'Doelsysteem')
df_doelsysteem.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Artefact,"[DB08, DB34, DC05, DC09, DC11, DC112, DC13, DC...",20690,20,Doelsysteem
1,Standplaats,[MAGAZIJN],16814,1,Doelsysteem
2,Aardewerk,"[DB34, DC05, DC112, DC20, DC21, DC22, DC36, DC...",14091,9,Doelsysteem
3,Plaatsing,[MAGAZIJN],11268,1,Doelsysteem
4,Doos,"[DB34, DC11, DC112, DC18, DC21, DC93, DC97, MA...",5574,8,Doelsysteem


## Analyse van alle gegevens

Hieronder volgt een overzicht van alle gegevens die in de vier fases beschikbaar zijn. Zo kunnen fouten in de conversie worden opgespoord. 

In [7]:


def getOverview(OnWhat): 
    lst_countColumns = [OnWhat + '_singlestore', OnWhat + '_singlestoreclean', OnWhat + '_doelsysteem']
    
    df = df_singlestore[['Soort', 'Projecten', OnWhat]].merge(df_singlestoreclean[['Soort', OnWhat]], on=['Soort'], how='outer', suffixes=("_singlestore", "_singlestoreclean"))
    df = df.merge(df_doelsysteem[['Soort', OnWhat]], on=['Soort'], how='outer', suffixes=("_singlestoreclean", "_doelsysteem"))

    df = df.sort_values(OnWhat+ '_singlestore', ascending=False) 
    df.rename(columns={'Aantal_Records': 'Aantal_Records_doelsysteem'}, inplace=True)
    df = df.merge(df_soorten_meta, on=['Soort'], how='outer')
    df[lst_countColumns] = df[lst_countColumns].fillna(0)
    df[lst_countColumns] = df[lst_countColumns].astype(int, errors='ignore')

    df = df.style.bar(subset=lst_countColumns, color='#5fba7d', vmax=20000)
    
    return df

getOverview('Aantal_Records')

,Soort,Projecten,Aantal_Records_singlestore,Aantal_Records_singlestoreclean,Aantal_Records_doelsysteem
0,Standplaats,['MAGAZIJN'],16814,16814,16814
1,Aardewerk,"['DC22', 'DC05', 'DC97', 'DB34', 'DC21', 'DC36', 'DC20', 'DC72', 'DC112']",14091,14091,14091
2,Plaatsing,['MAGAZIJN'],11268,11268,11268
3,Doos,['MAGAZIJN'],5546,5574,5574
4,Vondst,"['DC97', 'DC24_STADSKANTOOR', 'DC154', 'DC112', 'DC11', 'DC93', 'DC22', 'DB34', 'DC18', 'DC179', 'DC20']",5442,5538,5538
5,Foto,[nan],3227,3227,0
6,Spoor,"['DC179', 'DC97', 'DC24_STADSKANTOOR', 'DC112', 'DC93', 'DC154', 'DB34']",2560,2781,2781
7,Project,['DELF-IT'],1680,1680,1680
8,Hout,"['DC23', 'DC05', 'DC112', 'DC20', 'DB34', 'DC93', 'DC22', 'DC11', 'DC21', 'DC13', 'DC36', 'DC18', 'DC09', 'DC97']",1062,1062,1062
9,Glas,"['DC97', 'DC93', 'DB34', 'DC154']",922,922,922


## Detailanalyse SingleStore naar Doelsysteem

In [8]:
df = df_singlestore.merge(df_doelsysteem, on=['Soort', 'Aantal_Records', 'Aantal_projecten'], how='outer', suffixes=("_singlestore", "_doelsysteem"))
df[df.Stage_singlestore.isnull() | df.Stage_doelsysteem.isnull() ]

,Soort,Projecten_singlestore,Aantal_Records,Aantal_projecten,Stage_singlestore,Projecten_doelsysteem,Stage_doelsysteem
3,Doos,[MAGAZIJN],5546,1,SingleStore,NaN,NaN
4,Vondst,"[DC97, DC24_STADSKANTOOR, DC154, DC112, DC11, ...",5442,11,SingleStore,NaN,NaN
5,Foto,[nan],3227,1,SingleStore,NaN,NaN
6,Spoor,"[DC179, DC97, DC24_STADSKANTOOR, DC112, DC93, ...",2560,7,SingleStore,NaN,NaN
19,Vlak,"[DC24_STADSKANTOOR, DC154, DC179]",65,3,SingleStore,NaN,NaN
21,Put,"[DC154, DC179, DC24_STADSKANTOOR]",23,3,SingleStore,NaN,NaN
23,Artefact,NaN,20690,20,NaN,"[DB08, DB34, DC05, DC09, DC11, DC112, DC13, DC...",Doelsysteem
24,Doos,NaN,5574,8,NaN,"[DB34, DC11, DC112, DC18, DC21, DC93, DC97, MA...",Doelsysteem
25,Vondst,NaN,5538,22,NaN,"[DB34, DB8, DC11, DC112, DC13, DC14, DC154, DC...",Doelsysteem
26,Spoor,NaN,2781,7,NaN,"[DB34, DC112, DC154, DC179, DC24_STADSKANTOOR,...",Doelsysteem
